In [1]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from gen_mat import *
from block_rect_maxvol import *
import re
import os
from matplotlib import cm
from mva_test import *
import itertools
from ipywidgets import interactive, interact, widgets
%matplotlib inline
#matplotlib notebook
os.environ['OMP_NUM_THREADS'] = '6'
print (os.environ['OMP_NUM_THREADS'])

6


In [2]:
num_expr = 1 # Adding this num to output fn for distinguish between experiments  (not used for now)
nder = 2 # Dimension
ToLoadSet = False # whether to load settigns form file

functions = [f_rosenbrock, f_sincos, f_roots, f_gauss]
cut_radiuses = [None, 0.125, 0.15, 0.2]
inital_points_distribs = ['random', 'LHS']

if ToLoadSet:
    from sandbox_bm_approx_settings import *
else:
    dir_str = './'
    #dir_str = os.path.join(os.environ['HOME'], 'work/res/bm-big/')
    num_points_for_big_matrix = 12000 # number of points for big matrix
    max_row =  30                    # Maximum number of points taken in experiments
    max_expansion = 20             # number of columns in big matrix in (nder+1) units
    min_expansion = 3              # minimal number of columns in experiments in (nder+1) units
    

    n_test = 50000    # points on test grid (for calculating error on final step)
    poly = cheb       # used polinomials
    Polar_domain = False


#####
dir_pdf = os.path.join(dir_str, "pdf")
try:
    os.makedirs(dir_pdf)
except:
    pass



ToTakePointsFromFile = False # (not used for now)

# ---------------------------------
p_size = (nder+1)*max_row #number of rows in big matrix

### generating test points
points_test = test_points_gen(n_test, nder, distrib='random') 


#contents of file sandbox_bm_approx_settings.py
_="""
from gen_mat import cheb


dir_str = '.'
num_points_for_big_matrix = 50000 # number of points for big matrix
max_row = 50                      # Maximum number of points taken in experiments
max_expansion = 10                 # number of columns in big matrix in (nder+1) units
min_expansion = 1                 # minimal number of columns in experiments in (nder+1) units

n_test = 50000    # points on test grid (for calculating error on final step)
poly = cheb       # used polynomials
"""

### Testing the new most general environment

In [ ]:
### evaluating test
for inital_points_distrib in inital_points_distribs:
    points_fn = 'taken_points_{}'.format(inital_points_distrib)
    x = test_points_gen(num_points_for_big_matrix, nder, distrib=inital_points_distrib)
    if Polar_domain:
        x = domain_erase(x,7)
    A = GenMat(p_size, x, poly=poly, debug=False, pow_p=1)
    A = matrix_prep(A, nder+1)
    
    np.savez(os.path.join(dir_str, points_fn), x=x, points_test=points_test)
    
    
    
    fn_pre_pdf = "distrib={}".format(inital_points_distrib)
    for cut_radius in cut_radiuses:
        f = open(os.path.join(dir_str, "distrib={}_radius={}".format(inital_points_distrib, cut_radius) + '.txt'), "w")
        for expansion in range(min_expansion, max_expansion):
                    for N_rows_ex in range(max_row, expansion, -1): # It's not the way people do...
                        N_rows = N_rows_ex*(nder+1)
                        fnpdf = os.path.join(dir_pdf, fn_pre_pdf + "_expansion={}_N_rows_ex={}.pdf".format(expansion, N_rows_ex))
                        try:
                            taken_points = test_bm(A, x,nder, expansion, N_rows, cut_radius = cut_radius,to_save_pivs=N_rows_ex==max_row, 
                                                       fnpdf=fnpdf)
                        except SingularError as err:
                            print ('not full column rank with expansion={}, N_rows_ex={}, err={}'.format(
                                                                expansion, N_rows_ex, err.value)) 
                            #continue
                            break


                        taken_points.tofile(f, sep=" ")
                        f.write("_Nrows={}_expans={}\n".format(N_rows, expansion))
                        f.flush()


        f.close()


In [3]:
def file_extraction(Filepath, new_extr = True):
    if new_extr:
        srch = re.compile(r'([\d\s]+)_Nrows=(\d+)_expans=(\d+)')
        fnd = srch.findall(open(Filepath, 'r').read())
        return tuple(np.array(i) for i in zip(*[(int(i1), int(i2), [int(p) for p in im1.strip().split(' ') if len(p) > 0])
                                                for im1, i1, i2 in fnd]))
    else:
        srch = re.compile(r'([\d\s]+)_error=([\+\-\d\.eE]+)_Nrows=(\d+)_expans=(\d+)')
        fnd = srch.findall(open(Filepath, 'r').read())
        return tuple(np.array(i) for i in zip(*[(float(i0), int(i1), int(i2), [int(p) for p in im1.strip().split(' ') if len(p) > 0])
                                                for im1, i0, i1, i2 in fnd]))
def DataToMesh(error, N_row, N_col, *args):
    row_s = sorted(list(set(N_row)))
    col_s = sorted(list(set(N_col)))
    data = {(N_row[i], N_col[i]) : e for i, e in enumerate(error)}
    
    res = np.empty((len(row_s), len(col_s)), dtype=float)
    for i, r in enumerate(row_s):
        for j, c in enumerate(col_s):
            try:
                res[i,j] = data[(r, c)]
            except:
                res[i,j] = np.nan
    X, Y = np.meshgrid(row_s, col_s)
    return res.T, X, Y

def PlotError_3D(N_row, N_col, error_ext, log_it=False):
    error, N_row, N_col = DataToMesh(error_ext, N_row, N_col)
    
    if log_it:
        error = np.log10(error)
    
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_surface(N_row, N_col, error, edgecolor='black', linewidth=0.5, cmap = cm.Spectral)
    # ax.legend()
    ax.set_xlabel('N_rows', fontsize=10)
    ax.set_ylabel('N_columns')
    plt.show()

def PlotError(fn, error_ext, cols, rows,k, log_it=False):
    ndim = 3
    col_start = cols[0]
    print col_start
    error = error_ext
    N_row, N_col,_ = file_extraction(fn, new_extr=True)
    if log_it:
        error = np.log10(error)
    if cols is not None:
        if type(cols) is not list:
            cols = [cols]
        for i in cols:
            indx = np.where(N_col == i)
            fig = plt.figure()
            plt.plot(N_row[indx], error[indx])
            plt.xlabel('N_rows', fontsize=10)
            plt.ylabel('Error')
            fnpdf = 'err(rows)_column={}.pdf'.format(i)
            plt.grid(True)
            plt.savefig(fnpdf)
            plt.close(fig)
           
    if rows is not None:
        if type(rows) is not list:
            rows = [rows]
        for row in rows:
            indx_r = np.where(N_row == row)
            fig = plt.figure()
            plt.plot(N_col[indx_r], error[indx_r])
            plt.xlabel('N_cols', fontsize=10)
            plt.ylabel('Error')
            fnpdf = 'err(cols)_row={}.pdf'.format(row)
            plt.grid(True)
            plt.savefig(fnpdf)
            plt.close(fig)
            
            
def plot_error_slice(fn, error_ext, slices, rad, log_it=False, to_floor = True, adjust = True, N_adjust = 0):
    ndim = 3
    error = error_ext
    if log_it:
        error = np.log10(error)
    N_row, N_col,_ = file_extraction(fn, new_extr=True)
    cols = np.arange(N_col[0], N_col[-1])
    fig = plt.figure() 
    if adjust == False:
        for k in slices:
            indx = [] 
            for i in cols:
                if i == 5:
                    continue
                ix = np.where(N_col == i)[0]
                if to_floor:
                    loc_indx = np.where(N_row[ix]/ndim == np.floor(k*i))[0]
                else:
                    loc_indx = np.where(N_row[ix]/ndim == np.ceil(k*i))[0]

                try:### to get absolute index - make a summation 
                    indx.append(loc_indx[0] + ix[0])
                except:
                    break

            plt.plot(N_col[indx], error[indx], label = "k={}".format(k))
    else:
        
        for k in N_adjust:
            indx = []
            for i in cols:
                if i == 5:
                    continue
                ix = np.where(N_col == i)[0]    
                loc_indx = np.where(N_row[ix] == (i + k)*ndim)[0]
                
                try:
                    indx.append(loc_indx[0] + ix[0])
                except:
                    continue
            plt.plot(N_col[indx], error[indx], label = "k={}".format(k))
            
    plt.legend()
    plt.xlabel('N_rows', fontsize=10)
    plt.ylabel('Error')
    fnpdf = 'err(rows_div_cols)_tofloor={}_adjust={}_rad={}.pdf'.format(to_floor, adjust,rad)
    plt.grid(True)
    plt.savefig(fnpdf)
    plt.close(fig)

### Experimental data processing

In [4]:
### Data extraction
dir_str = './'
dir_points = os.path.join(dir_str, "bm-big4/")
points_distrib = 'LHS'
rad = "0.25"
taken_points = np.load(dir_points + "taken_points_" + points_distrib + ".npz")
x = taken_points['x']
points_test = taken_points['points_test']

In [160]:
### .txt processing
functions = [f_branin]

N_row, N_col, p_indices = file_extraction(dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt")

### old one parsing
#pr_error, N_row, N_col,p_inx  = file_extraction(dir_points+"func=Roots_poly=cheb_distrib=random.txt", new_extr=False)

In [161]:
error = np.empty(N_row.shape)
error_LHS = np.empty(N_row.shape)
error_random = np.empty(N_row.shape)
error_sobol = np.empty(N_row.shape)
Leb = np.empty(N_row.shape)
x_LHS = test_points_gen(num_points_for_big_matrix, nder, distrib='LHS')
x_random = test_points_gen(num_points_for_big_matrix, nder, distrib='random')
x_sobol = GenSobol( N = num_points_for_big_matrix, dim = 2, rng=(-1, 1) )
if type(functions) is not list:
        functions = [functions]
for j, p in enumerate(p_indices):
    ValsandNorms = MakeValsAndNorms(functions, points_test)
    Leb[j], error[j] = LebesgueConst(x[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
    _, error_LHS[j] = LebesgueConst(x_LHS[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
    _, error_random[j] = LebesgueConst(x_random[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
    _, error_sobol[j] = LebesgueConst(x_sobol[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)

In [ ]:
PlotError_3D(N_row, N_col, error_random, True)

In [ ]:
plot_error_slice(dir_points + "distrib=" + points_distrib + "_radius=0.15.txt", error,[1.2, 1.3,1.4,1.45, 1.5,1.6,1.7,1.8,1.9],0.25, False, True, False, [1,2,3,4,5])

### Random points plot

In [6]:
from sobol_lib import *
def GenSobol( N = 200, dim = 2, seed = 0, rng=(-1.0, 1.0) ):
    res = np.zeros((N, dim), dtype=float)
    rng_d = rng[1] - rng[0]
    for i in xrange(N):
        res[i, :], seed = i4_sobol ( dim, seed )
        res[i, :] = res[i, :]*rng_d + rng[0]

    return res

In [162]:
def multi_error_slice(fn, k, error, error_LHS, error_random, error_sobol, log_it=False, adjust = False):
    N_row, N_col,_ = file_extraction(fn, new_extr=True)
    ndim = 3
    cols = np.arange(N_col[0], N_col[-1])
    fig = plt.figure(figsize=(10,6)) 
    to_floor = True
    if log_it:
            error = np.log10(error)
            error_LHS = np.log10(error_LHS)
            error_random = np.log10(error_random)
            error_sobol = np.log10(error_sobol)
    if adjust == False:
        indx = [] 
        for i in cols:
            if i == 5:
                continue
            ix = np.where(N_col == i)[0]
            if to_floor:
                loc_indx = np.where(N_row[ix]/ndim == np.floor(k*i))[0]
            else:
                loc_indx = np.where(N_row[ix]/ndim == np.ceil(k*i))[0]

            try:### to get absolute index - make a summation 
                indx.append(loc_indx[0] + ix[0])
            except:
                break
    else:
        indx = []
        for i in cols:
            if i == 5:
                continue
            ix = np.where(N_col == i)[0]    
            loc_indx = np.where(N_row[ix] == (i + k)*ndim)[0]

            try:
                indx.append(loc_indx[0] + ix[0])
            except:
                continue
    N_col = 3*N_col            
    plt.plot(N_col[indx], error[indx], label = "BMaxvol")
    plt.plot(N_col[indx], error_LHS[indx], label = "LHS")
    plt.plot(N_col[indx], error_random[indx], label = "Random")
    plt.plot(N_col[indx], error_sobol[indx], label = "Sobol")
    plt.legend()
    plt.xlabel('Number of monoms', fontsize=10)
    plt.ylabel('Logscaled error')
    plt.grid(True)
    fnpdf = 'err(monoms)_slice={}.pdf'.format(k)
    plt.grid(True)
    plt.savefig(fnpdf)
    plt.close(fig)
    
    
def multi_error_straight(fn, error, error_LHS, error_random, error_sobol, col, rows,log_it=False):
    ndim = 3
    N_row, N_col,_ = file_extraction(fn, new_extr=True)
    if log_it:
        error = np.log10(error)
        error_LHS = np.log10(error_LHS)
        error_random = np.log10(error_random)
        error_sobol = np.log10(error_sobol)
    if col is not None:
        cols = np.arange(N_col[0], N_col[-1])
        #if type(cols) is not list:
           # cols = [cols]
        for i in cols:
            indx = np.where(N_col == i)
            fig = plt.figure()
            plt.plot(N_row[indx], error[indx], label = 'BMaxvol')
            plt.plot(N_row[indx], error_LHS[indx], label = "LHS")
            plt.plot(N_row[indx], error_random[indx], label = "Random")
            plt.plot(N_row[indx], error_sobol[indx], label = "Sobol")
            plt.xlabel('N_rows', fontsize=10)
            plt.ylabel('Error')
            plt.legend()
            fnpdf = 'err(rows)_column={}.pdf'.format(i)
            plt.grid(True)
            plt.savefig(fnpdf)
            plt.close(fig)
           
    if rows is not None:
        if type(rows) is not list:
            rows = rows.tolist()
        for row in rows:
            indx = np.where(N_row == row)
            fig = plt.figure()
            plt.plot(N_col[indx], error[indx], label = 'BMaxvol')
            plt.plot(N_col[indx], error_LHS[indx], label = "LHS")
            plt.plot(N_col[indx], error_random[indx], label = "Random")
            plt.plot(N_col[indx], error_sobol[indx], label = "Sobol")
            plt.xlabel('N_cols', fontsize=10)
            plt.ylabel('Error')
            plt.legend()
            fnpdf = 'err(cols)_row={}.pdf'.format(row)
            plt.grid(True)
            plt.savefig(fnpdf)
            plt.close(fig)    
        

In [165]:
multi_error_slice(dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt", 1.3, error, error_LHS, error_random, error_sobol, log_it=True, adjust = False)

In [166]:
multi_error_straight(dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt", error, error_LHS, error_random, error_sobol, 1, np.arange(24,84,3,dtype=int),log_it=True)

## Separate B-maxvol test

In [ ]:
num_points_for_big_matrix = 1200
cut_radiuses = [None, 0.3, 0.025, 0.25]
expansion = 12
row_exp = 30
nder = 2
p_size = expansion*(nder+1)
distr = 'LHS'
x = test_points_gen(num_points_for_big_matrix, nder, distrib=distr)
A = GenMat(p_size+3, x, poly=cheb, debug=False, pow_p=1)
A = matrix_prep(A, nder+1)

In [ ]:
A2 = np.copy(A)
A1 = np.copy(A)
A = A1[:,:p_size]
fn_pre_pdf = "distrib={}".format(distr)
for cut_radius in cut_radiuses:
    print (np.linalg.norm(A2 - A1,2))
    f = open(os.path.join(dir_str, "distrib={}_radius={}".format(distr, cut_radius) + '.txt'), "w")       
    fnpdf = os.path.join(dir_pdf, fn_pre_pdf + "_expansion={}_N_rows_ex={}.pdf".format(p_size, row_exp))
    try:
        if cut_radius == None:
            to_erase = None
        else:    
            erase_init(point_erase, x, nder, r = cut_radius)
            to_erase = point_erase
        pivs = rect_block_maxvol(A, nder, Kmax = row_exp*(nder+1), max_iters=100, rect_tol = 0.05, tol = 0.0, debug = False, to_erase = to_erase)
    except SingularError as err:
        print ('not full column rank with expansion={}, N_rows_ex={}, err={}'.format(
                                            expansion, row_exp, err.value)) 
        #continue
        break
    cut_piv = pivs[:row_exp*(nder+1)]
    taken_indices = cut_piv[::(nder+1)] // (nder+1)
    

    l_bound = np.amin(x, 0)
    u_bound = np.amax(x, 0)
    delta = (u_bound - l_bound)/20.0
    fig = plt.figure()
    plt.xlim(l_bound[0] - delta[0], u_bound[0] + delta[0])
    plt.ylim(l_bound[1] - delta[1], u_bound[1] + delta[1])
    plt.plot(x[pivs[::(nder+1)] // (nder+1), 0], x[pivs[::(nder+1)] // (nder+1), 1], 'b^')
    plt.plot(x[taken_indices, 0], x[taken_indices, 1], 'r^')
    plt.grid(True)

  
    fnpdf = 'columns={}_rows={}_rad={}.pdf'.format(expansion*(nder+1), row_exp*(nder+1), cut_radius)

    plt.savefig(fnpdf)
    plt.close(fig)
    taken_indices.tofile(f, sep=" ")
    f.write("_Nrows={}_expans={}\n".format(row_exp*(nder+1), expansion))
    f.flush()    

In [60]:
col_exp = 18
i = np.where(N_col == col_exp)[0][0]
taken_indices = p_indices[i]
def parameter_plot(N_points):
    l_bound = np.amin(x, 0)
    u_bound = np.amax(x, 0)
    delta = (u_bound - l_bound)/20.0
    fig = plt.figure()
    plt.xlim(l_bound[0] - delta[0], u_bound[0] + delta[0])
    plt.ylim(l_bound[1] - delta[1], u_bound[1] + delta[1])
    plt.plot(x[taken_indices[:N_points], 0], x[taken_indices[:N_points], 1], 'r^')
    plt.grid(True)
    
controls = {'N_points': widgets.IntSlider(
    min=col_exp, max=max_row, step=1, value=2, continuous_update=False, description='$Points$')
}

In [61]:
interact(parameter_plot, **controls);

interactive(children=(IntSlider(value=18, continuous_update=False, description=u'$Points$', max=30, min=18), O…

In [ ]:
N_row, N_col, p_indices = file_extraction(dir_points + "distrib=" + points_distrib + "_radius=None.txt")

In [ ]:
col_exp = 6
i = np.where(N_col == col_exp)[0][0]
taken_indices = p_indices[i]

In [ ]:
interact(parameter_plot, **controls);